# Performing EDA on Venmo data

In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pymongo
import json
import datetime
import pickle
import functions as fn

Data has to be exported from a Mongo DB

In [2]:
# Retrieve the venmo transaction collection from the MongoDB
venmo = fn.collection()

In [3]:
# Count all transactions in the venmo data
venmo.count()

/Users/jjherranzsarrion/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


7076585

In [4]:
# Inspect the first transaction
venmo.find_one()

{'_id': ObjectId('5bb7bdce1bed297da9fcb251'),
 'mentions': {'count': 0, 'data': []},
 'comments': {'count': 0, 'data': []},
 'date_created': datetime.datetime(2018, 7, 26, 18, 48, 10),
 'audience': 'public',
 'type': 'payment',
 'authorization': None,
 'id': '2532209455978775150',
 'payment': {'note': 'for utilities',
  'date_created': '2018-07-26T18:48:10',
  'id': '2532209455660008361',
  'amount': None,
  'date_authorized': None,
  'status': 'settled',
  'actor': {'first_name': 'Nicole',
   'last_name': 'Andrews',
   'phone': None,
   'username': 'Nicole-Andrews-23',
   'id': '2206066431492096327',
   'profile_picture_url': 'https://s3.amazonaws.com/venmo/no-image.gif',
   'identity': None,
   'trust_request': None,
   'email': None,
   'is_blocked': False,
   'display_name': 'Nicole Andrews',
   'friend_status': None,
   'about': ' ',
   'is_group': False,
   'date_joined': '2017-05-02T19:00:12',
   'is_active': True,
   'friends_count': None},
  'merchant_split_purchase': None,
  

In [ ]:
#pass in initial_5pct function to generate a pickle with the first ~350k transactions
#initial_5pct = fn.initial_5pct(venmo)

In [5]:
# load the above mentioned pickle
with open('initial_5pct_transactions.pkl', 'rb') as f:
    initial_5pct = pickle.load(f)

In [ ]:
first = venmo.find_one()

In [ ]:
transaction_df = []
transaction = {}
keys = ['mentions', 'likes', 'comments','app']
payment_keys = (['amount', 'note', 'action', 'status', 'date_created', 'date_reminded',
                 'id', 'date_authorized', 'merchant_split_purchase', 'audience', 'date_completed'])
payment_inner_keys = ['target', 'actor']
target_keys = ['redeemable_target', 'merchant', 'phone', 'email', 'type']
actor_keys = ["username", "friends_count", "is_active", "display_name", "friend_status", "email",
              "first_name", "identity", "last_name", "is_blocked", "about", "profile_picture_url", "id",
              "phone", "trust_request", "date_joined", "is_group"]    

for key, val in first.items():
    if key in keys:
        for subkeys, subvals in val.items():
            unpacked = f'{key}_{subkeys}'
            transaction[unpacked] = subvals
    elif key == 'payment':
        for payment_subkeys, payment_subvals in val.items():
            if payment_subkeys in payment_keys:
                payments_unpacked = f'{key}_{payment_subkeys}'
                transaction[payments_unpacked] = payment_subvals
            elif payment_subkeys in payment_inner_keys:
                for payment_target_actor_subkeys, payment_target_actor_subvalues in payment_subvals.items():
                    if payment_target_actor_subkeys in target_keys:
                        payment_target_subkeys_unpacked = f'{key}_{payment_subkeys}_{payment_target_actor_subkeys}'
                        transaction[payment_target_subkeys_unpacked] = payment_target_actor_subvalues
                    elif payment_target_actor_subkeys in actor_keys:
                        payment_actor_subkeys_unpacked = f'{key}_{payment_subkeys}_{payment_target_actor_subkeys}'
                        transaction[payment_actor_subkeys_unpacked] = payment_target_actor_subvalues
                    elif payment_target_actor_subkeys == 'user':
                        for payment_target_user_subkeys, payment_target_user_subvalues in payment_target_actor_subvalues.items():
                            payment_actor_user_subkeys_unpacked = (
                                f'{key}_{payment_subkeys}_{payment_target_actor_subkeys}_{payment_target_user_subkeys}'
                            )
                            transaction[payment_actor_user_subkeys_unpacked] = payment_target_user_subvalues
                    else:
                        continue
            else:
                continue
        continue
    else:
        transaction[key] = val
transaction_df.append(transaction.copy())

In [ ]:
transaction_df

## Inspecting payer information

In [6]:
len(initial_5pct)

353829

In [7]:
# Collect the unique ids for each payer in the intial_5pct of transactions
payer_ids = set()
for transaction in initial_5pct:
    actor = transaction['payment']['actor']
    actor_id = actor['id']
    payer_ids.add(actor_id)

In [8]:
f'The number of unique payers in the first 5% of transactions is {len(payer_ids)}'

'The number of unique payers in the first 5% of transactions is 329112'

In [16]:
# Transform user information into a dataframe
payers = []
payer_ids = set()
counter = 0
for transaction in initial_5pct:
    actor = transaction['payment']['actor']
    actor_id = actor['id']
    if actor_id in payer_ids:
        continue
    else:
        payer_ids.add(actor_id)
        payer = {}
        for key, val in transaction['payment']['actor'].items():
            payer[key] = val
        payers.append(payer.copy())

payers_df = pd.DataFrame(payers)
payers_df['payer'] = 1

In [17]:
payers_df.head()

,about,date_joined,display_name,email,first_name,friend_status,friends_count,id,identity,is_active,is_blocked,is_group,last_name,phone,profile_picture_url,trust_request,username,payer
0,,2017-05-02T19:00:12,Nicole Andrews,None,Nicole,None,None,2206066431492096327,None,True,False,False,Andrews,None,https://s3.amazonaws.com/venmo/no-image.gif,None,Nicole-Andrews-23,1
1,,2017-04-24T23:57:10,Christy Faw,None,Christy,None,None,2200417693859840681,None,True,False,False,Faw,None,https://s3.amazonaws.com/venmo/no-image.gif,None,Christy-Faw,1
2,,2017-12-19T22:56:09,Vladan Stankovic,None,Vladan,None,None,2373608382922752189,None,True,False,False,Stankovic,None,https://venmopics.appspot.com/u/v1/s/3c4263b0-...,None,Vladan-Stankovic-1,1
3,No Short Bio,2015-04-24T20:33:48,Elizabeth Giordano,None,Elizabeth,None,None,1670504276557824171,None,True,False,False,Giordano,None,https://venmopics.appspot.com/u/v1/s/215499bc-...,None,egiordano,1
4,,2016-05-24T17:22:23,Brenda Carobini,None,Brenda,None,None,1957419122950144676,None,True,False,False,Carobini,None,https://venmopics.appspot.com/u/v1/s/92d9c5b4-...,None,Brenbini,1


In [18]:
# Investigate the dataframe
payers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329112 entries, 0 to 329111
Data columns (total 18 columns):
about                  329111 non-null object
date_joined            329111 non-null object
display_name           329112 non-null object
email                  0 non-null object
first_name             329112 non-null object
friend_status          0 non-null object
friends_count          0 non-null object
id                     329112 non-null object
identity               0 non-null object
is_active              329112 non-null bool
is_blocked             329112 non-null bool
is_group               329112 non-null bool
last_name              329112 non-null object
phone                  0 non-null object
profile_picture_url    329112 non-null object
trust_request          0 non-null object
username               329111 non-null object
payer                  329112 non-null int64
dtypes: bool(3), int64(1), object(14)
memory usage: 38.6+ MB


In [19]:
# Isolate nulls to see them more clearly
payers_df.isna().sum()

about                       1
date_joined                 1
display_name                0
email                  329112
first_name                  0
friend_status          329112
friends_count          329112
id                          0
identity               329112
is_active                   0
is_blocked                  0
is_group                    0
last_name                   0
phone                  329112
profile_picture_url         0
trust_request          329112
username                    1
payer                       0
dtype: int64

In [20]:
# Inspect the null value for about, looks like it could be the same row as for date_joined and username.
payers_df.loc[payers_df['about'].isna()]

,about,date_joined,display_name,email,first_name,friend_status,friends_count,id,identity,is_active,is_blocked,is_group,last_name,phone,profile_picture_url,trust_request,username,payer
294315,None,None,Colleen Methvin,None,Colleen,None,None,2407520882655232799,None,False,False,False,Methvin,None,https://s3.amazonaws.com/venmo/no-image.gif,None,None,1


It would be insteresting to see if said user has made any transactions

In [21]:
# Given that it is we are going to drop said value.
payers_df.drop(axis=0, index=294315, inplace=True)

In [22]:
# Drop columns that only have null values
null_cols = ['email', 'friend_status', 'friends_count', 'identity', 'phone', 'trust_request']
payers_df.drop(labels = null_cols, axis=1, inplace=True)

In [23]:
payers_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329111 entries, 0 to 329111
Data columns (total 12 columns):
about                  329111 non-null object
date_joined            329111 non-null object
display_name           329111 non-null object
first_name             329111 non-null object
id                     329111 non-null object
is_active              329111 non-null bool
is_blocked             329111 non-null bool
is_group               329111 non-null bool
last_name              329111 non-null object
profile_picture_url    329111 non-null object
username               329111 non-null object
payer                  329111 non-null int64
dtypes: bool(3), int64(1), object(8)
memory usage: 26.1+ MB


In [24]:
payers_df['about'].value_counts()

                                                           316122
No Short Bio                                                11265
No short bio                                                   64
\n                                                             12
 \n                                                             8
                                                                7
No Short Bio\n                                                  7
💸                                                               6
💰💰💰                                                             6
$$$                                                             6
Hi                                                              6
💰                                                               6
 .                                                              5
 hi                                                             5
hi                                                              4
😎         

In [26]:
# Converting the date joined object into a datetime.datetime field
payers_df['date_joined'] = pd.to_datetime(payers_df['date_joined'], format='%Y-%m-%dT%H:%M:%S')

In [29]:
payers_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329111 entries, 0 to 329111
Data columns (total 12 columns):
about                  329111 non-null object
date_joined            329111 non-null datetime64[ns]
display_name           329111 non-null object
first_name             329111 non-null object
id                     329111 non-null object
is_active              329111 non-null bool
is_blocked             329111 non-null bool
is_group               329111 non-null bool
last_name              329111 non-null object
profile_picture_url    329111 non-null object
username               329111 non-null object
payer                  329111 non-null int64
dtypes: bool(3), datetime64[ns](1), int64(1), object(7)
memory usage: 26.1+ MB


In [31]:
payers_df['is_active'].value_counts()

True    329111
Name: is_active, dtype: int64

In [30]:
payers_df['is_blocked'].value_counts()

False    329111
Name: is_blocked, dtype: int64

In [32]:
payers_df['is_group'].value_counts()

False    329083
True         28
Name: is_group, dtype: int64

In [37]:
# Investigating whether or not display_name is just a sum of first_name and last_name
payers_df['new_name'] = (payers_df['first_name'] + ' ' + payers_df['last_name'])

In [42]:
len(payers_df) - sum(payers_df['display_name'] == payers_df['new_name'])

35

In [40]:
# Investigating the cases in which display name is not equal to the combination of first_name and last_name
payers_df.loc[payers_df['display_name'] != payers_df['new_name']]

,about,date_joined,display_name,first_name,id,is_active,is_blocked,is_group,last_name,profile_picture_url,username,payer,new_name
1822,,2017-06-30 16:04:20,Murray SC Boys U14 Orange,,2248739683041280434,True,False,True,,https://s3.amazonaws.com/venmo/placeholder-ima...,murrayscboysu14,1,
5643,,2017-08-11 18:12:52,Psyclones Valor,,2279244956696576648,True,False,True,,https://venmopics.appspot.com/u/v1/s/b9f644d5-...,PsyclonesValor,1,
16604,,2017-03-09 06:00:12,"Special Guest App, Inc.","Special Guest App, Inc.",2166535955152897010,True,False,False,Account,https://s3.amazonaws.com/venmo/no-image.gif,SpecialGuestAppInc,1,"Special Guest App, Inc. Account"
17467,,2017-07-11 05:00:19,APRL,APRL,2256378005553152080,True,False,False,Account,https://s3.amazonaws.com/venmo/no-image.gif,APRL,1,APRL Account
19578,,2017-02-28 21:08:37,BlachConstruction,,2160470194651136096,True,False,True,,https://venmopics.appspot.com/u/v10/s/8b36720d...,BlachConst-laura-bold,1,
40155,,2018-01-25 18:53:12,Sigma Nu - Gamma Upsilon,,2400302804238336942,True,False,True,,https://venmopics.appspot.com/u/v1/s/aff4c5f8-...,SigmanuGU,1,
44974,,2017-10-12 01:28:41,UCSD Zor,,2323675630010368549,True,False,True,,https://venmopics.appspot.com/u/v1/s/df5539d8-...,ucsd-zor9,1,
71521,,2018-02-12 16:06:17,HHSLPA,,2413264755490816512,True,False,True,,https://venmopics.appspot.com/u/v1/s/624bcc60-...,HHSLPA,1,
74033,,2018-04-21 16:31:08,725 Church,,2462562012626944171,True,False,True,,https://s3.amazonaws.com/venmo/placeholder-ima...,seventwentyfive,1,
77117,,2018-02-02 19:10:32,Dog City Disco,,2406109734240256323,True,False,True,,https://venmopics.appspot.com/u/v1/s/db519f86-...,dogcitydisco,1,


In [43]:
payers_df.drop('new_name', axis=1, inplace=True)

Another revelation has just been made, although no null values seem to appear in the first and last name columns, there are blank values in it. Lets investigate this further.

In [48]:
payers_df.loc[payers_df['first_name'] == '']

,about,date_joined,display_name,first_name,id,is_active,is_blocked,is_group,last_name,profile_picture_url,username,payer
1822,,2017-06-30 16:04:20,Murray SC Boys U14 Orange,,2248739683041280434,True,False,True,,https://s3.amazonaws.com/venmo/placeholder-ima...,murrayscboysu14,1
5643,,2017-08-11 18:12:52,Psyclones Valor,,2279244956696576648,True,False,True,,https://venmopics.appspot.com/u/v1/s/b9f644d5-...,PsyclonesValor,1
19578,,2017-02-28 21:08:37,BlachConstruction,,2160470194651136096,True,False,True,,https://venmopics.appspot.com/u/v10/s/8b36720d...,BlachConst-laura-bold,1
40155,,2018-01-25 18:53:12,Sigma Nu - Gamma Upsilon,,2400302804238336942,True,False,True,,https://venmopics.appspot.com/u/v1/s/aff4c5f8-...,SigmanuGU,1
44974,,2017-10-12 01:28:41,UCSD Zor,,2323675630010368549,True,False,True,,https://venmopics.appspot.com/u/v1/s/df5539d8-...,ucsd-zor9,1
71521,,2018-02-12 16:06:17,HHSLPA,,2413264755490816512,True,False,True,,https://venmopics.appspot.com/u/v1/s/624bcc60-...,HHSLPA,1
74033,,2018-04-21 16:31:08,725 Church,,2462562012626944171,True,False,True,,https://s3.amazonaws.com/venmo/placeholder-ima...,seventwentyfive,1
77117,,2018-02-02 19:10:32,Dog City Disco,,2406109734240256323,True,False,True,,https://venmopics.appspot.com/u/v1/s/db519f86-...,dogcitydisco,1
78015,,2016-10-31 23:15:46,Rescue Paws USA - UGA,,2073561103597568287,True,False,True,,https://venmopics.appspot.com/u/v2/s/378e2682-...,rescuepawsUGA,1
78261,,2016-11-01 14:05:28,PeteyGreeneDC,,2074008904269824481,True,False,True,,https://venmopics.appspot.com/u/v1/s/07d13939-...,PeteyGreeneDC,1


All columns that are groups don't have either a first_name or last_name associated with it.

In [49]:
payers_df['profile_picture_url'].value_counts()

https://s3.amazonaws.com/venmo/no-image.gif                                   63039
https://s3.amazonaws.com/venmo/placeholder-image/groups-placeholder.svg           6
https://venmopics.appspot.com/u/v1/s/096f397c-f058-416d-9413-33ad8da08107         1
https://venmopics.appspot.com/u/v1/s/f0d6c472-b059-4fdb-bce0-cbb5a13df284         1
https://venmopics.appspot.com/u/v1/s/96fc568e-a242-4c5d-be08-13e790cc3993         1
https://venmopics.appspot.com/u/v1/s/0a0acdf8-b706-4a2f-949a-8b6f9dfa9c10         1
https://venmopics.appspot.com/u/v1/s/4e1dd111-f556-48a7-9eee-4148ed3444f7         1
https://graph.facebook.com/v2.10/1242261862473506/picture?type=square             1
https://venmopics.appspot.com/u/v1/s/42238df3-e75f-4d91-8bd9-6de63e91d369         1
https://venmopics.appspot.com/u/v1/s/9bc77816-bc39-4075-a8b9-6ee0f00a06a3         1
https://venmopics.appspot.com/u/v2/s/7ad5e98d-50f0-4eec-bc26-87ef6ae5fd05         1
https://venmopics.appspot.com/u/v1/f/293fb157-966a-4799-9554-7f15d03edf20   

In [50]:
payers_df['username'].value_counts()

Omar-Sanchez-74         2
jdbooth2                1
Jade-Salzano            1
Ashley-Bargsley         1
beth-rich               1
Chrystina-Andrade       1
RebeccaRedmond          1
John-Gamboa-jag         1
sampankow               1
FranFrutos              1
valencianator           1
jimmy-jam0              1
jeffrey-erickson        1
Megan-Albertini         1
Drew-Martin-19          1
MgoCrabtree             1
James-Yu-15             1
Leina-Nguyen            1
Chancy-Holbrook         1
James-Pimentel-91       1
lacinea                 1
Ricardo1                1
Michael-Lugossy         1
Raymond-Alas            1
Kevin-Radziewicz        1
Qbanbailarin            1
KayKayCorona            1
Hudson-Lorfing          1
Jack-Mangold            1
jared-dupes             1
                       ..
muffysmith              1
Elliott-Moore-5         1
Robert-Hosford          1
kelsey-hutchison        1
cody-comiskey           1
GrantRussell            1
Jofin-Mathai            1
Maya-Familet

In [51]:
payers_df.loc[payers_df['username'] == 'Omar-Sanchez-74']

,about,date_joined,display_name,first_name,id,is_active,is_blocked,is_group,last_name,profile_picture_url,username,payer
253027,,2018-07-28 23:44:02,Omar Sanchez,Omar,2533807919988736204,True,False,False,Sanchez,https://venmopics.appspot.com/u/v1/s/c8fe11a8-...,Omar-Sanchez-74,1
263251,,2018-07-29 02:24:07,Omar Sanchez,Omar,2533888492568576142,True,False,False,Sanchez,https://s3.amazonaws.com/venmo/no-image.gif,Omar-Sanchez-74,1
